# Deep XOR #

The goal of this notebook is to illustrate, step by step, the modelling of an [XOR gate](https://en.wikipedia.org/wiki/Exclusive_or) using a simple neural network trained via gradient descent.

To have a better understanding of neural networks, I decided go through the derivation of all the necessary equations, and implement them with [numpy](http://www.numpy.org/). 
Hopefully it is helpful to other practitioners too. Life is sharing (openly on github) :)

(The whole thing is inspired by Section 6.2 of the absolutely fantastic [The Deep Learning Book](http://www.deeplearningbook.org/), and [this](http://cs231n.github.io/neural-networks-case-study/#net))

In [1]:
import numpy as np  # That's all we need :)

## The Data ##

We randomly generate a handful of examples to later train our model. 
There are only 4 different unique input combinations to XOR:
* $0 \oplus 0 = 0$
* $0 \oplus 1 = 1$
* $1 \oplus 0 = 1$
* $1 \oplus 1 = 0$

$N$ observations of pairs of ones and zeroes are generated and stored in $X \in \mathbb{N}^{N \times 2}$, and passed through the XOR operation to obtain the target set of labels $Y \in \mathbb{N}^{N \times 2}$, which we encode as one-hot vectors.

In [2]:
def gen_data(N):
    """Generates an XOR dataset using one-hot vector encoding.
    
    Parameters
    ----------
    N: int
        Number of observations to be generated.
    
    Returns
    -------
    X: np.ndarray((N, 2))
        Matrix containing an observation per row, representing the input
        to the XOR operation (observations).
    Y: np.ndarray((N, 2))
        Result of the XOR operation for each row, encoded with 
        one-hot vector per row (targets).
    """
    X = np.random.randint(0, 2, size=(N, 2))
    Y = np.zeros((N, 2))
    Y[np.arange(N), [int(np.logical_xor(x[0], x[1])) for x in X]] = 1
    return X, Y

def xor_print(X, Y, N):
    """Prints out an XOR dataset.
    
    Parameters
    ----------
    X: np.ndarray((N, 2))
        Observations.
    Y: np.ndarray((N, 2))
        Targets.
    N: int
        Number of observations to be printed.
    """
    [print(str(x[0]) + ' XOR ' + str(x[1]) + ' = ' + str(np.argmax(y))) 
     for x, y in zip(X[:N], Y[:N])]

# Create some training data
N = 100
X, Y = gen_data(N)
xor_print(X, Y, 10)

1 XOR 1 = 0
0 XOR 1 = 1
0 XOR 1 = 1
0 XOR 0 = 0
0 XOR 1 = 1
0 XOR 0 = 0
1 XOR 0 = 1
1 XOR 0 = 1
0 XOR 0 = 0
0 XOR 0 = 0


## The Model ##

To approximate the XOR function we need a non-linear model. 
Since neural networks are so [hot](http://i.imgur.com/FNoToht.jpg) nowardays, we design a feedforward neural network with a single hidden layer.

The network has two output units $\hat{y}_1$ and $\hat{y}_2$, representing the likelihood of having a 0 or a 1, respectively, thus framing the problem as **classification**. The network diagram may look as follows:
![Feedforward Neural Network2](./diagram_sm.png)

Where the weights $W_1, W_2$ and basis $b_1, b_2$ are the coefficients to be learned, $\mathbf{x} = \{x_1, x_2\}$ is the input, $\mathbf{h} = \{h_1, h_2\}$ is the content of the hidden layer, and $\mathbf{\hat{y}} = \{\hat{y}_1, \hat{y}_2\}$ is the output of the network.

We use a rectified linear unit (ReLU) for the activation of the hidden layer. Formally:

$$\mathbf{h} = \mbox{max}\{\mathbf{0}, \mathbf{x}^T W_1 + b_1\}$$

For the output layer, we use the standard softmax function, which is the common activation used for classification problems:

$$\mathbf{\hat{y}} = \frac{\exp(\mathbf{h}^T W_2 + b_2)}{\sum_j^2\exp(\mathbf{h}^T W_2 + b_2)_j}$$

### Multiple Inputs ###

If we focus on multiple observations $X = \{\mathbf{x}_1^T, \mathbf{x}_2^T, \ldots, \mathbf{x}_N^T\}$ instead of a single one $\mathbf{x}_i$ we can rewrite the equations as follows:

$$H = \mbox{max}\{\mathbf{0}, X W_1 + b_1\}$$

$$\hat{Y} = \frac{\exp(H W_2 + b_2)}{\sum_j^2\exp(H W_2 + b_2)_j}$$

where the biases $b_1, b_2$ are broadcasted across their respective inputs.

We can now implement a function that computes a multiple input forward pass of the network using numpy:

In [3]:
def forward_pass(X, W1, W2, b1, b2):
    """Computes a forward pass of the network. Once the coefficients are trained,
    this should compute the XOR on all x \in X.
    
    Parameters
    ----------
    X: np.ndarray((N, 2))
        Inputs to the network.
    W1: np.ndarray((2, n_hidden))
        Set of weights for the first layer.
    W2: np.ndarray((n_hidden, 2))
        Set of weights for the second layer.
    b1: float
        Bias for the first layer.
    b3: float
        Bias for the second layer.
        
    Returns
    -------
    H: np.ndarray((N, n_hidden))
        Content of the hidden layer.
    Y_est: np.ndarray((N, 2))
        Output of the network, the two columns represent the likelihood of
        having a 0 or a 1, in this order.
    """
    # Hidden layer forward pass
    H = np.maximum(0, np.dot(X, W1) + b1)  # ReLU activation
    
    # Second layer forward pass without activation
    linear = np.dot(H, W2) + b2
    
    # Softmax activation (output)
    Y_est = np.exp(linear) / np.sum(np.exp(linear), axis=1, keepdims=True)  # [N x 2]
    return H, Y_est

## Weights and Biases Initialization ##

We can initialize our training coefficients randomly. Notice how it would be straightforward to include more hidden units in our hidden layer. Feel free to play with this parameter, it seems to converge faster with 3 or 4 units. But for now, let's stick to our original model of 2 hidden units.

In [4]:
# Input -> Hidden layer weights and bias
n_hidden = 2
W1 = np.random.random(size=(2, n_hidden))
b1 = np.zeros((1, n_hidden)) # bias

# Hidden -> Output layer weights and bias
K = 2  # Either 0 or 1
W2 = np.random.random(size=(n_hidden, K))
b2 = np.zeros((1, K)) # bias

## Loss Function ##

Next step is to define an appropriate differentiable loss function that allows us to train our network. The loss should be small when we are obtaining successful results with the current parameters, and high otherwise.

The **cross-entropy** function is commonly used in classification problems, since its derivative is quite simple and yet successfully captures the cost of the network for a given set of parameters.

The average cross-entropy is defined as:

$$L = - \frac{1}{N}\sum_{i}^N \mathbf{y}_i \odot \log(\mathbf{\hat{y}}_i) $$

where $\odot$ is the element-wise product.

Let's quickly inspect this function. If we would have the perfect prediction (e.g., $\mathbf{y}_i = \{0, 1\}$ and $\mathbf{\hat{y}}_i = \{0, 1\}$), then $L_i$ would be 0, as desired. Whereas, if we would have a completely wrong one (e.g., $\mathbf{y}_i = \{0, 1\}$ and $\mathbf{\hat{y}}_i = \{1, 0\}$), then we would get $L_i = \infty$.

In [5]:
def compute_loss(Y, Y_est):
    """Computes the averate cross-entropy loss.
    
    Parameters
    ----------
    Y: np.ndarray((N, 2))
        Target labels (XOR correct results).
    Y: np.ndarray((N, 2))
        Estimated XOR values.
        
    Returns
    -------
    loss: float
        Loss value: the average cross-entropy
    """
    N = Y.shape[0]
    return - np.sum(np.log(np.sum(Y_est * Y, axis=1))) / float(N)

In [6]:
n_epochs = 30000
step = 0.01
for n_epoch in range(n_epochs):
    # Forward pass of the network, catching the two layers' outputs
    H, Y_est = forward_pass(X, W1, W2, b1, b2)
    
    # Loss: average cross-entropy loss
    loss = compute_loss(Y, Y_est)
    
    # Accuracy
    acc = np.sum(Y[np.arange(N), np.argmax(Y_est, axis=1)]) / float(N)
    
    # Printout
    if n_epoch % 1000 == 0:
        print("\tEpoch %d: loss %f\t acc %.1f%%" % (n_epoch, loss, (acc * 100)))
        
    # Gradient of the second (output) layer
    dpreds = np.copy(Y_est)
    dpreds[np.where(Y == 1)] -= 1
    dpreds /= float(N)
    
    # Gradient of the hidden layer
    dhidden = np.dot(dpreds, W2.T)
    dhidden[H <= 0] = 0  # ReLU backprop
    
    # Backpropagation of output layer
    dW2 = np.dot(H.T, dpreds)
    db2 = np.sum(dpreds, axis=0, keepdims=True)
    
    # Backprop of hidden layer
    dW1 = np.dot(X.T, dhidden)
    db1 = np.sum(dhidden, axis=0, keepdims=True)
  
    # Update step
    W1 -= step * dW1
    b1 -= step * db1
    W2 -= step * dW2
    b2 -= step * db2
    
    # Generate some new training data (this may help to avoid falling into local minima)
    X, Y = gen_data(N)

	Epoch 0: loss 0.706469	 acc 52.0%
	Epoch 1000: loss 0.649812	 acc 50.0%
	Epoch 2000: loss 0.521621	 acc 67.0%
	Epoch 3000: loss 0.331217	 acc 100.0%
	Epoch 4000: loss 0.186399	 acc 100.0%
	Epoch 5000: loss 0.111837	 acc 100.0%
	Epoch 6000: loss 0.071910	 acc 100.0%
	Epoch 7000: loss 0.055673	 acc 100.0%
	Epoch 8000: loss 0.038435	 acc 100.0%
	Epoch 9000: loss 0.029984	 acc 100.0%
	Epoch 10000: loss 0.026018	 acc 100.0%
	Epoch 11000: loss 0.022100	 acc 100.0%
	Epoch 12000: loss 0.019636	 acc 100.0%
	Epoch 13000: loss 0.018815	 acc 100.0%
	Epoch 14000: loss 0.015000	 acc 100.0%
	Epoch 15000: loss 0.012655	 acc 100.0%
	Epoch 16000: loss 0.011165	 acc 100.0%
	Epoch 17000: loss 0.011422	 acc 100.0%
	Epoch 18000: loss 0.010205	 acc 100.0%
	Epoch 19000: loss 0.008463	 acc 100.0%
	Epoch 20000: loss 0.007935	 acc 100.0%
	Epoch 21000: loss 0.009081	 acc 100.0%
	Epoch 22000: loss 0.007036	 acc 100.0%
	Epoch 23000: loss 0.007746	 acc 100.0%
	Epoch 24000: loss 0.006187	 acc 100.0%
	Epoch 25000: lo

In [5]:
X_test, Y_test = gen_data(N)
_, Y_est = forward_pass(X_test, W1, W2, b1, b2)
acc = np.sum(Y_test[np.arange(N), np.argmax(Y_est, axis=1)]) / float(N)
print("Accuracy in Test set: %.2f%%" % (acc * 100))

Accuracy in Test set: 100.00%


In [37]:
# Normal equations, solving with these four examples:
X = np.array([[1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]])  # Added the bias as ones in the first column
y = np.array([0, 1, 1, 0])
b, w1, w2 = np.dot(np.linalg.inv(np.dot(X.T, X)), np.dot(X.T, y))